In [1]:
!pip install numpy pandas
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import re
import string
import pandas as pd
import nltk
# import contractions
# from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

nyt_df = pd.read_csv("nyt.csv")

# CLEAN TEXT

def clean(text):
  text = text.lower()
  text = re.sub(r'\d+', '', text)
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = re.sub(r'\W', ' ', text)
  return text

nyt_df['text'] = nyt_df['text'].apply(clean)

# tokenization
nyt_df['text'] = nyt_df['text'].apply(word_tokenize)

# stopwords

stop_words = set(stopwords.words('english'))

def remove_stop(text):
  return [word for word in text if word not in stop_words]

nyt_df['text'] = nyt_df['text'].apply(remove_stop)

# lemmatization

lemmatizer = WordNetLemmatizer()

def lem(text):
  return [lemmatizer.lemmatize(word) for word in text]

nyt_df['text'] = nyt_df['text'].apply(lem)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
# word embedding using glove

import numpy as np


# load
def load_glove(path):
  embeddings = {}
  with open(path, 'r', encoding = 'utf-8') as f:
    for line in f:
      word_vector = line.strip().split()
      word = word_vector[0]
      vector = np.array(word_vector[1:], dtype=np.float32)
      embeddings[word] = vector
  return embeddings

glove_path = "glove.6B.50d.txt"
glove = load_glove(glove_path)

# converting the vectors

def text_to_vector(text, glove, dim=50):
  vector = []
  for word in text:
    if word in glove:
      vector.append(glove[word])
    else:
      vector.append(np.zeros(dim))
  return vector

nyt_df['vector'] = nyt_df['text'].apply(lambda text: text_to_vector(text, glove))

print(nyt_df.head())



             author                                               text  stars  \
0      Le Bernardin  [tell, people, city, great, sushi, counter, gu...      2   
1           Yoshida  [show, minute, early, pm, reservation, tadashi...      2   
2  Four Twenty Five  [announcement, chef, jeangeorges, vongerichten...      2   
3          Bungalow  [done, butterchicken, era, indian, restaurant,...      2   
4             Penny  [chef, tend, load, first, restaurant, every, h...      2   

                                              vector  \
0  [[0.27569, 0.11783, 0.37882, -0.49723, 0.76888...   
1  [[-0.034772, 0.64145, -0.37352, 0.20804, 0.204...   
2  [[0.71427, -0.076561, -0.00010032, 0.61367, 0....   
3  [[0.33076, -0.4387, -0.32163, -0.4931, 0.10254...   
4  [[-0.12078, 0.59621, -1.4258, 0.16769, 0.69497...   

                                         pad_vectors  
0  [[0.2756899893283844, 0.11783000081777573, 0.3...  
1  [[-0.03477200120687485, 0.6414499878883362, -0...  
2  [[0.7142

In [8]:
# padding

max_length = 300


def padding(seq):
  if len(seq) >= max_length:
        return np.array(seq[:max_length])
  elif len(seq) < max_length:
    padded = [np.zeros(50) for _ in range(max_length - len(seq))]
    return np.array(seq + padded)
  else:
    print("else")
    return [np.zeros(50) for _ in range(max_length)]

nyt_df["pad_vectors"] = nyt_df["vector"].apply(padding)

# todo:
# 1. make a vector for each word in the text list
# 2. split into

nyt_df.head()

,author,text,stars,vector,pad_vectors
0,Le Bernardin,"[tell, people, city, great, sushi, counter, gu...",2,"[[0.27569, 0.11783, 0.37882, -0.49723, 0.76888...","[[0.2756899893283844, 0.11783000081777573, 0.3..."
1,Yoshida,"[show, minute, early, pm, reservation, tadashi...",2,"[[-0.034772, 0.64145, -0.37352, 0.20804, 0.204...","[[-0.03477200120687485, 0.6414499878883362, -0..."
2,Four Twenty Five,"[announcement, chef, jeangeorges, vongerichten...",2,"[[0.71427, -0.076561, -0.00010032, 0.61367, 0....","[[0.7142699956893921, -0.07656099647283554, -0..."
3,Bungalow,"[done, butterchicken, era, indian, restaurant,...",2,"[[0.33076, -0.4387, -0.32163, -0.4931, 0.10254...","[[0.33076000213623047, -0.43869999051094055, -..."
4,Penny,"[chef, tend, load, first, restaurant, every, h...",2,"[[-0.12078, 0.59621, -1.4258, 0.16769, 0.69497...","[[-0.12077999860048294, 0.5962100028991699, -1..."


In [23]:
nyt_df.to_pickle("nyt_df.pkl")